In [3]:
import torch

NUM_CLASS = 100

In [4]:
target = torch.zeros(NUM_CLASS, dtype=torch.float)
target

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.])

In [7]:
target[[2, 3, 4, 5]]=1.
target

tensor([0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

prepare (fake) data:

In [11]:
import os
import unibox as ub


IMG_ROOT = "data"

# 格式是{number}.webp
imgs = ub.traverses(IMG_ROOT, ub.IMG_FILES, relative_unix=True)
for i in range(len(imgs)):
    _img = imgs[i]
    os.system(f"mv {IMG_ROOT}/{_img} {IMG_ROOT}/{i}.webp")

In [13]:
# labels: 格式是 list[  [number, extension, class]  ]  
import random

labels = []
for i in range(len(imgs)):
    # random pick 1~5 classes
    num_classes = random.randint(1, 5)
    _classes = random.sample(range(NUM_CLASS), num_classes)
    labels.append([i, "webp", [_classes]])

labels[:5]

[[0, 'webp', [[47, 70, 13, 49]]],
 [1, 'webp', [[60, 3]]],
 [2, 'webp', [[96, 16]]],
 [3, 'webp', [[21, 98, 29, 38]]],
 [4, 'webp', [[10]]]]

In [14]:
# save labels as a json
import json

with open("fake_labels.json", "w") as f:
    json.dump(labels, f)

In [21]:
!python train_ac.py --imgs_train ./data --imgs_val ./data --label_train ./fake_labels.json --label_val ./fake_labels.json

/home/ubuntu/dev/mld-debug/src_files/models/caformer/metaformer_baselines.py:843: UserWarning: Overwriting poolformerv2_s12 in registry with src_files.models.caformer.metaformer_baselines.poolformerv2_s12. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def poolformerv2_s12(pretrained=False, **kwargs):
/home/ubuntu/dev/mld-debug/src_files/models/caformer/metaformer_baselines.py:859: UserWarning: Overwriting poolformerv2_s24 in registry with src_files.models.caformer.metaformer_baselines.poolformerv2_s24. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def poolformerv2_s24(pretrained=False, **kwargs):
/home/ubuntu/dev/mld-debug/src_files/models/caformer/metaformer_baselines.py:875: UserWarning: Overwriting poolformerv2_s36 in registry with src_files.models.caformer.metaformer_baselines.poolformerv2_s36. This is because the name being registered conflicts 

In [19]:
!pip install -U timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 33.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: timm
    Found existing installation: timm 0.9.7
    Uninstalling timm-0.9.7:
      Successfully uninstalled timm-0.9.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
open-clip-torch 2.20.0 requires protobuf<4, but you have protobuf 4.25.2 which is incompatible.
tagit 0.2.17 requires huggingface-hub<0.17.0,>=0.16.4, but you have huggingface-hub 0.20.3 which is incompatible.
